In [59]:
# import des librairies requises
import pandas as pd
import unicodedata
import re
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from stop_words import get_stop_words

In [32]:
# lecture des différents csv dans les 3 variables 
df1 = pd.read_csv("./../data/offers.csv")
#df2 = pd.read_csv("./../data/organizations.csv")
#df3 = pd.read_csv("./../data/recruiters.csv")

# df1
# df2
# df3

In [33]:
df_offers = df1[df1.contract_id.notnull()]

In [34]:
# Remplacer les valeurs NA avec ""

df_offers['description'] = [BeautifulSoup(text, "html.parser").get_text() for text in df_offers['description'].fillna("")]

C:\Users\Antoine\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
import unidecode

In [36]:
def my_preprocessor(text):
    return unidecode.unidecode(''.join([letter for letter in text if not letter.isdigit()]).strip().lower())

In [37]:
# Vectorisation

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer


#stop words fr dans stop_words
stop_wordsFR = get_stop_words('french')

vectorizer = TfidfVectorizer(
    ngram_range=(1, 3),
    analyzer="word",
    binary=False,
    max_features=6000,
    stop_words = stop_wordsFR,
preprocessor=my_preprocessor)


In [38]:
#tfidf fit de cleaned_description

vectorizer.fit(df_offers.description)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=6000, min_df=1,
        ngram_range=(1, 3), norm='l2',
        preprocessor=<function my_preprocessor at 0x0909EF18>,
        smooth_idf=True,
        stop_words=['a', 'ai', 'aie', 'aient', 'aies', 'ait', 'alors', 'as', 'au', 'aucun', 'aura', 'aurai', 'auraient', 'aurais', 'aurait', 'auras', 'aurez', 'auriez', 'aurions', 'aurons', 'auront', 'aussi', 'autre', 'aux', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', '...ça', 'étaient', 'étais', 'était', 'étant', 'état', 'étiez', 'étions', 'été', 'étés', 'êtes', 'être'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [39]:
X = vectorizer.transform(df_offers.description)

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test= train_test_split(X, df_offers.contract_id, test_size=0.30)

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
clf = LogisticRegression()

In [44]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [45]:
from collections import Counter

In [46]:
Counter(clf.predict(X_test))

Counter({1.0: 2789, 2.0: 68, 3.0: 138})

In [47]:
Counter(y_test)

Counter({1.0: 2449, 2.0: 265, 3.0: 246, 4.0: 32, 6.0: 2, 5.0: 1})

In [64]:
confusion_matrix(y_test, clf.predict(X_test))

array([[2448,    0,    1,    0,    0,    0],
       [ 197,   67,    1,    0,    0,    0],
       [ 122,    0,  124,    0,    0,    0],
       [  19,    1,   12,    0,    0,    0],
       [   1,    0,    0,    0,    0,    0],
       [   2,    0,    0,    0,    0,    0]], dtype=int64)

In [48]:
xArray = X[:10].toarray()

In [49]:
xArray

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [51]:
from sklearn.pipeline import Pipeline

In [28]:
pipeline = Pipeline(steps=[
    ('text_extraction', vectorizer),
    ('classifier', LogisticRegression())
])

Pipeline(memory=None,
     steps=[('text_extraction', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=6000, min_df=1,
        ngram_range=(1, 2), norm='l2',
        preprocessor=<funct...
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None))])

In [ ]:
#pipeline.fit(....)

In [45]:
#pipeline.predict(.....)

SyntaxError: invalid syntax (<ipython-input-45-ac3aa0488c75>, line 1)